In [1]:
# Generic imports
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import os
from sys import platform
from importlib import reload
import sys
import scipy.optimize as optimization
sys.path.append('../')
import plotly.graph_objects as go
from plotly.offline import iplot

# Local imports
import stlstuff as sls
import imagestuff as ims
import statstuff as sts
import retrievestuff as rs

CUDA not available.


In [2]:
%matplotlib notebook

In [4]:
%ls ../crystals/2023-07-03/"case4.0 (calibration)"

Basal Facet Crosscut Case 4.0.png
Boxes.nml
Calibration.nml
GNBF_1_makextlvecs.ipynb
GNBF_2_calibrate_from_Xtlvecs.ipynb
GNBF_3_retrieve.ipynb
GNBF_4_flattenandfilter.ipynb
GNBF_5_analyzeroughness_of_flattened_npz.ipynb
GNBF_6_grid2stlwskirt.ipynb
SEMimages/
Segments1.jpg
Segments1.nml
Segments1_compr.npz
Segments1_compr_flat.npz
Segments1_compr_flat_filt.npz
Segments1_retrieved.jpg
Segments1_retrieved.npz
Segments1_retrievedwskirt.stl
Segments2.jpg
Segments2.nml
Segments2_compr.npz
Segments2_compr_flat.npz
Segments2_compr_flat_filt.npz
Segments2_retrieved.jpg
Segments2_retrieved.npz
Segments3.jpg
Segments3.nml
Segments3_compr.npz
Segments3_compr_flat.npz
Segments3_compr_flat_filt.npz
Segments3_retrieved.jpg
Segments3_retrieved.npz
Segments4.jpg
Segments4.nml
Segments4_compr.npz
Segments4_compr_flat.npz
Segments4_compr_flat_filt.npz
Segments4_retrieved.jpg
Segments4_retrieved.npz
Segments5.jpg
Segments5.nml
Segments5_compr.npz
Segments5_compr_flat.npz
Segments5_compr_flat_filt.npz
Segm

The next cell has parameters that might change from crystal to crystal

In [22]:
# Loading in the compressed data
Dirname = "../crystals/2023-07-03/case4.0 (calibration)/"
Segmentname = 'Segments1'
Flattenedfilename = Dirname+Segmentname+'_compr_flat_filt.npz'; print(Flattenedfilename)

# Histogram accumulation: indexing starts at 0 for the upper-left, down, then across. Set to [] for all segments
accumlist = [] 

# Parameters for binning
Z2minforhist = 0.0
Z2maxforhist = .05 #can set max depending on how much roughness expected
Z2offset = 0.001
nbins_max = 5
levels = 3

../crystals/2023-07-03/case4.0 (calibration)/Segments1_compr_flat_filt.npz


In [29]:
# Derivative names
Roughnessfilename = Flattenedfilename[0:-4]+'_roughness.jpg'; print(Roughnessfilename)
flattenedfile = np.load(Flattenedfilename); print(Flattenedfilename)
xgridtot = flattenedfile['xgridtot']
ygridtot = flattenedfile['ygridtot']
zgridtot = flattenedfile['zgridtot']
nsegments = (len(xgridtot)); print(nsegments)

../crystals/2023-07-03/case4.0 (calibration)/Segments1_compr_flat_filt_roughness.jpg
../crystals/2023-07-03/case4.0 (calibration)/Segments1_compr_flat_filt.npz
1
[]


In [9]:
%whos

Variable            Type        Data/Info
-----------------------------------------
Dirname             str         ../crystals/2023-07-03/case4.0 (calibration)/
Flattenedfilename   str         ../crystals/2023-07-03/ca<...>ents1_compr_flat_filt.npz
Roughnessfilename   str         ../crystals/2023-07-03/ca<...>r_flat_filt_roughness.jpg
Segmentname         str         Segments1
Z2maxforhist        float       0.05
Z2minforhist        float       0.0
Z2offset            float       0.001
accumlist           list        n=0
axes3d              module      <module 'mpl_toolkits.mpl<...>lkits/mplot3d/axes3d.py'>
flattenedfile       NpzFile     <numpy.lib.npyio.NpzFile object at 0x16e5266d0>
go                  module      <module 'plotly.graph_obj<...>aph_objects/__init__.py'>
ims                 module      <module 'imagestuff' from<...>eatest/../imagestuff.py'>
iplot               function    <function iplot at 0x16e3bed40>
levels              int         3
nbins_max           int        

In [10]:
# Arrays for accumulating 
counts_list = []
meanZ2_list = []
Z2flat_list = []

# First-guess for Weibull fitting (sigma2W, etaW)
x0 = np.array([.1,.9])

# This suppresses warnings when lots of graphics windows are open
import matplotlib
matplotlib.rc('figure', max_open_warning = 0)

# graph config
full_render = False

crosscut_layer = 0
graph_x = True
graph_y = False



# Looping over segments
for isegment in range(nsegments):

    # Pull out the next segment
    sollast = zgridtot[isegment]
    xgrid = xgridtot[isegment]
    ygrid = ygridtot[isegment]
    Ny, Nx = np.shape(sollast); #print(sollast.shape)
    Ntot = np.size(sollast)
    dx = xgrid[0,1]-xgrid[0,0]; #print('dx =', dx)
    dy = ygrid[1,0]-ygrid[0,0]; #print('dy =', dy)
        
# Reporting
    print('')
    print('****Working on segment', isegment)

    # 3D Render
    if full_render:
        fig1 = plt.figure()
        ax = fig1.add_subplot(111, projection='3d')
        ax.plot_surface(xgrid, ygrid, sollast)
        title = Flattenedfilename+' #'+str(isegment)
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        ax.set_title(title)
        ax.view_init(azim=-120,elev=22)
        
    # Graph 2D Crosscut
    if graph_x:
        plt.figure()
        plt.plot(xgrid[0], sollast[0])
        title = Flattenedfilename+ "Y Axis Crosscut Layer: " + str(crosscut_layer)
        plt.xlabel('x')
        plt.ylabel('z')
        plt.title(title)
        plt.grid(True)
    
    if graph_y:
        # Inverts y grid
        y_invert = []
        for i in range(len(ygrid)):
            y_invert.append(ygrid[i][crosscut_layer])
        
        # Inverts sollast
        sollast_invert = []
        for i in range(len(sollast)):
            sollast_invert.append(sollast[i][crosscut_layer])
            
        #Graph 2D Crosscut
        plt.figure()
        plt.plot(y_invert, sollast_invert)
        title = Flattenedfilename+ "X Axis Crosscut Layer: " + str(crosscut_layer)
        plt.xlabel('y')
        plt.ylabel('z')
        plt.title(title)
        plt.grid(True)
        
            
            
    
        
    
            
        
#         # Normalize the distribution function, report stats
#         integral_rho = np.trapz(counts, bins)
#         print ('std dev of height = ', np.std(sollast))
#         print('integral = ', integral_rho)
#         counts = counts/integral_rho
#         error = error/integral_rho
#         print('error = ', error)

#         # This is for calculating the errors in fitting (and graphing)
#         countsplus = counts+error; #print(countsplus)
#         countsminus = counts**2/countsplus; #print(countsminus)

#         # Graph the probability
#         if Show_intermediate_graphics:
            
#             # Using plotly
#             plot1 = go.Scatter(x=bins, y=np.log(counts), marker=dict(color='blue'),mode='markers')
#             plot2 = go.Scatter(x=bins, y=np.log(countsplus), marker=dict(color='black'),mode='markers')
#             plot3 = go.Scatter(x=bins, y=np.log(countsminus), marker=dict(color='black'),mode='markers')
#             iplot([plot1,plot2,plot3])

#         # Eliminate entries greater than a threshold (not sure if this is necessary)
#         ikeep = np.argwhere(Z2flat < Z2maxforhist)
#         Z2flat_new = np.squeeze(Z2flat[ikeep])

#         # Specify errors in a log space
#         errors = countsplus/counts; #print(errors)
        
#         # Do a linear fit
#         f_linear = lambda x, a, b: a + b*x   # function to fit
#         solution, corr = optimization.curve_fit(f_linear, bins+Z2offset, np.log(counts), [5, -100], errors)
#         a_linear_optimized = solution[0]; print('a_linear_optimized', a_linear_optimized)
#         b_linear_optimized = solution[1]; print('b_linear_optimized', b_linear_optimized)
#         sigma2G = -1/b_linear_optimized
#         sigmaG = np.sqrt(sigma2G); print('sigmaG', sigmaG)

#         # Do a quadratic fit
#         f_quadratic = lambda x, a, b, c: a + b*x +c*x**2  # function to fit
#         solution, corr = optimization.curve_fit(f_quadratic, bins+Z2offset, np.log(counts), [5, -100, 0], errors)
#         a_quadratic_optimized = solution[0]; print('a_quadratic_optimized', a_quadratic_optimized)
#         b_quadratic_optimized = solution[1]; print('b_quadratic_optimized', b_quadratic_optimized)
#         c_quadratic_optimized = solution[2]; print('c_quadratic_optimized', c_quadratic_optimized)
#         sigma2W = -1/b_quadratic_optimized
#         sigmaW = np.sqrt(sigma2W); print('approximate sigmaW', sigmaW)
        
#         # Calculate the Gaussian fit
#         myGaussian = sts.Gaussian(Z2theory,sigma2G)
        
#         # Calculate the Weibill fit -- 
#         # Note that this is a kluge: it should be replaced by myWeibull = sts.Weibull(Z2theory,sigma2W,etaW)
#         myWeibull = np.exp(f_quadratic(Z2theory,a_quadratic_optimized,b_quadratic_optimized,c_quadratic_optimized))

#         # Graph data with the best-fit probabilities
#         if Show_intermediate_graphics:
#             # Using matplotlib
#             plt.figure()
#             plt.semilogy(bins, counts, 'ok')
#             plt.semilogy(bins, countsplus, '+k')
#             plt.semilogy(bins, countsminus,'+k')
#             plt.title(title)
#             plt.xlabel(r'$Z^{2}$')
#             plt.ylabel(r'$\rho$')
#             plt.grid(True)
#             plt.semilogy(Z2theory, myGaussian, 'b',label = 'Gaussian with $\sigma_G$ = ' + str(sigmaG)[0:5])
#             plt.semilogy(Z2theory, myWeibull, 'g--',label = 'Quadratic')
#             plt.legend()

#     # Accumulate if we want
#     if isegment in accumlist:
#         counts_list.append(counts)
#         meanZ2_list.append(meanZ2)
#         Z2flat_list.append(Z2flat)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [17]:
# Sum up the accumulated information
Z2flat_total = []
for i in Z2flat_list:
    for j in i:
        Z2flat_total.append(j)
Z2flat_total = np.array(Z2flat_total)
Z2flat_new = np.squeeze(Z2flat_total)

# Get the probability distribution in Z2
counts, bins, meanZ2, error = rs.getrhoofz2flat(Z2flat,nbins,Z2bins,levels)
print('meanZ2 = ', meanZ2)
print('statsigma = ', np.sqrt(meanZ2))

# Normalize the distribution function
integral_rho = np.trapz(counts, bins)
print('integral = ', integral_rho)
counts = counts/integral_rho
error = error/integral_rho
countsplus = counts+error; #print(countsplus)
countsminus = counts**2/countsplus; #print(countsminus)

Original =  1088
4 0 272 [272   0   0   0]
4 1 272 [272   0   0   0]
4 2 272 [272   0   0   0]
4 3 272 [272   0   0   0]
ilevelp = 4
ilevelp, t = 4 2.7764451051977987
meanZ2 =  0.0005518364361246947
statsigma =  0.023491199120621635
integral =  0.0025


C:\Users\tiabo\AppData\Local\Temp/ipykernel_14244/3676951824.py:20: RuntimeWarning:

invalid value encountered in true_divide



In [18]:
# Specify errors in a log space
errors = countsplus/counts; #print(errors)

# Do a linear fit
f_linear = lambda x, a, b: a + b*x   # function to fit
solution, corr = optimization.curve_fit(f_linear, bins+Z2offset, np.log(counts), [5, -100], errors)
a_linear_optimized = solution[0]; print('a_linear_optimized', a_linear_optimized)
b_linear_optimized = solution[1]; print('b_linear_optimized', b_linear_optimized)

# Calculate the Gaussian parameter (sigmaG) from the best-fit slope, and the resulting PDF
sigma2G = -1/b_linear_optimized
sigmaG = np.sqrt(sigma2G); print('sigmaG', sigmaG)
myGaussian = sts.Gaussian(Z2theory,sigma2G)

# Do a quadratic fit
f_quadratic = lambda x, a, b, c: a + b*x +c*x**2  # function to fit
solution, corr = optimization.curve_fit(f_quadratic, bins+Z2offset, np.log(counts), [5, -100, 0], errors)
a_quadratic_optimized = solution[0]; print('a_quadratic_optimized', a_quadratic_optimized)
b_quadratic_optimized = solution[1]; print('b_quadratic_optimized', b_quadratic_optimized)
c_quadratic_optimized = solution[2]; print('c_quadratic_optimized', c_quadratic_optimized)

# Calculate Quadratic parameters (sigmaQ, etaQ) from the best-fit slope and curvature, and the resulting PDF
sigma2Q, etaQ = sts.bc_to_Q_parameters(b_quadratic_optimized,c_quadratic_optimized)
sigmaQ = sigma2Q**.5; print('sigmaQ', sigmaQ)
myQuadratic = sts.Quadratic(Z2theory,sigma2Q,etaQ)
myQuadratic *= counts[0]/myQuadratic[0] # This is a kluge until we get the normalization factor formula

# Calculate Weibull parameters (sigmaW, etaW) from the best-fit slope and curvature, and the resulting PDF
sigma2W, etaW = sts.bc_to_Weibull_parameters(b_quadratic_optimized,c_quadratic_optimized)
sigmaW = sigma2W**.5; print('sigmaW', sigmaW)
myWeibull = sts.Weibull(Z2theory,sigma2W,etaW)

# Graph data with the best-fit probabilities
plt.figure()
plt.semilogy(bins, counts, 'ok')
plt.semilogy(bins, countsplus, '+k')
plt.semilogy(bins, countsminus,'+k')
plt.title(title)
plt.xlabel(r'$Z^{2}$')
plt.ylabel(r'$\rho$')
plt.grid(True)
plt.semilogy(Z2theory, myGaussian, 'b',label = 'Gaussian with $\sigma_G$ = ' + str(sigmaG)[0:5])
plt.semilogy(Z2theory, myWeibull, 'g',label = \
             'Weibull with $\sigma_W$ = ' + str(sigmaW)[0:5] + ', $\eta_W$ = ' + str(etaW)[0:5])
# plt.semilogy(Z2theory, myQuadratic, 'r--',label = \
#              'Quadratic with $\sigma_Q$ = ' + str(sigmaQ)[0:5] + ', $\eta_Q$ = ' + str(etaQ)[0:5])
plt.legend()

C:\Users\tiabo\AppData\Local\Temp/ipykernel_14244/538829213.py:2: RuntimeWarning:

invalid value encountered in true_divide

C:\Users\tiabo\AppData\Local\Temp/ipykernel_14244/538829213.py:6: RuntimeWarning:

divide by zero encountered in log



ValueError: array must not contain infs or NaNs

In [ ]:
# # Save the roughness figure
# plt.savefig(Roughnessfilename)

In [ ]:
# Saving ith segment as a textfile.
saving_text_file = False
if saving_text_file:
    iSegment = 0
    zfilename = Segmentname+'_z_values.csv'; print(zfilename)
    xfilename = Segmentname+'_x_values.csv'; print(xfilename)
    yfilename = Segmentname+'_y_values.csv'; print(yfilename)
    Z2filename = Segmentname+'_Z2_values.csv'; print(Z2filename)
    print(np.shape(zgridtot))
    print(np.shape(Z2flat_list[iSegment]))

    myxgridtot = xgridtot[iSegment,0,:]
    myygridtot = ygridtot[iSegment,:,0]
    myzgridtot = zgridtot[iSegment,:,:]
    myZ2flat = Z2flat_list[iSegment]

    nxy = np.shape(myzgridtot)
    ny = nxy[0]
    nx = nxy[1]
    print(nxy)

    myZ2 = np.reshape(myZ2flat, (ny-1,nx-1)) # Z2 is 1 shorter in either dimension

    print(np.shape(myZ2))
    np.savetxt(zfilename, myzgridtot, delimiter = ',')
    np.savetxt(xfilename, myxgridtot, delimiter = ',')
    np.savetxt(yfilename, myygridtot, delimiter = ',')
    np.savetxt(Z2filename, Z2flat_list[iSegment], delimiter = ',')

    fig = go.Figure(data = go.Surface(x = myxgridtot[1:], y = myygridtot[1:], z = myZ2))
    fig.update_layout(scene = dict(xaxis_title = 'x',yaxis_title = 'y',zaxis_title = 'z'))

In [15]:
# Crosscut Graphs and degree calculation
print(zgridtot)
# which slice to cut
x_slice = 0
y_slice = 0

# find z
x_crosscut_z = zgridtot[x_slice,:]
y_crosscut_z = zgridtot[:,y_slice]
x = xseggrid[0][0,:]
y = yseggrid[0][:,0]


# x-Crosscut
plt.figure()
plt.plot(x, x_crosscut_z)
plt.grid()
plt.title("x-crosscut at slice " + str(x_slice))
plt.xlabel("x (micrometers)")
plt.ylabel("z (micrometers)")

rise_from_left = max(x_crosscut_z)-x_crosscut_z[0]
distance_from_left = x[np.argmax(x_crosscut_z)]-x[0]

rise_from_right = max(x_crosscut_z)-x_crosscut_z[-1]
distance_from_right = x[-1]-x[np.argmax(x_crosscut_z)]

radians_left = np.arctan2(rise_from_left, distance_from_left)
radians_right = np.arctan2(rise_from_right, distance_from_right)

degrees_left = radians_left * 180/np.pi
degrees_right = radians_right * 180/np.pi

print("Degrees change from left "+str(degrees_left))
print("Degrees change from right "+str(degrees_right))

# y-Crosscut
plt.figure()
plt.title("y-crosscut at slice " + str(y_slice))
plt.plot(y, y_crosscut_z)
plt.grid()
plt.xlabel("y (micrometers)")
plt.ylabel("z (micrometers)")

rise_from_left = max(y_crosscut_z)-y_crosscut_z[0]
distance_from_left = y[np.argmax(y_crosscut_z)]-y[0]

rise_from_right = max(y_crosscut_z)-y_crosscut_z[-1]
distance_from_right = y[-1]-y[np.argmax(y_crosscut_z)]

radians_left = np.arctan2(rise_from_left, distance_from_left)
radians_right = np.arctan2(rise_from_right, distance_from_right)

degrees_left = radians_left * 180/np.pi
degrees_right = radians_right * 180/np.pi

print("Degrees change from left "+str(degrees_left))
print("Degrees change from right "+str(degrees_right))

[]


IndexError: index 0 is out of bounds for axis 1 with size 0